<a href="https://colab.research.google.com/github/SahadStats/data-analytics-assignment/blob/main/DA_assignment_upliance_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# Cooking Platform Data Analysis


!pip install pandas numpy matplotlib seaborn openpyxl

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from google.colab import files



def upload_excel_file():
    """
    Upload Excel file in Google Colab
    """
    print("Please upload the Assignment.xlsx file")
    uploaded = files.upload()

    # Return the filename of the uploaded file
    return list(uploaded.keys())[0] if uploaded else None

def load_and_clean_data(filename):
    """
    Load datasets and perform initial cleaning
    """
    # Load datasets
    user_details = pd.read_excel('/content/Assignment.xlsx', sheet_name='UserDetails.csv')
    cooking_sessions = pd.read_excel('/content/Assignment.xlsx', sheet_name='CookingSessions.csv')
    order_details = pd.read_excel('/content/Assignment.xlsx', sheet_name='OrderDetails.csv')

    # Data Cleaning
    # Convert date columns to datetime
    user_details['Registration Date'] = pd.to_datetime(user_details['Registration Date'])
    cooking_sessions['Session Start'] = pd.to_datetime(cooking_sessions['Session Start'])
    cooking_sessions['Session End'] = pd.to_datetime(cooking_sessions['Session End'])
    order_details['Order Date'] = pd.to_datetime(order_details['Order Date'])

    # Handle missing values
    order_details['Rating'] = pd.to_numeric(order_details['Rating'], errors='coerce')

    # Remove potential duplicates
    user_details.drop_duplicates(subset=['User ID'], inplace=True)
    cooking_sessions.drop_duplicates(subset=['Session ID'], inplace=True)
    order_details.drop_duplicates(subset=['Order ID'], inplace=True)

    return user_details, cooking_sessions, order_details

def analyze_user_demographics(user_details, order_details):
    """
    Analyze user demographics and their relationship with orders
    """
    # Age group distribution
    user_details['Age Group'] = pd.cut(user_details['Age'],
                                       bins=[0, 25, 35, 45, 100],
                                       labels=['18-25', '26-35', '36-45', '45+'])

    # Merge demographics with order details
    user_order_analysis = user_details.merge(order_details, on='User ID')

    # Orders by age group
    age_group_orders = user_order_analysis.groupby('Age Group')['Amount (USD)'].agg(['count', 'mean'])
    age_group_orders.columns = ['Total Orders', 'Average Order Value']

    # Location-based analysis
    location_orders = user_order_analysis.groupby('Location')['Amount (USD)'].agg(['count', 'mean'])
    location_orders.columns = ['Total Orders', 'Average Order Value']

    return age_group_orders, location_orders

def analyze_cooking_sessions(cooking_sessions, order_details):
    """
    Analyze cooking session characteristics and their impact on orders
    """
    # Merge cooking sessions with order details
    session_order_data = cooking_sessions.merge(order_details, on=['Session ID', 'User ID'])


    # Session duration impact
    session_order_data['Session Duration'] = (session_order_data['Session End'] - session_order_data['Session Start']).dt.total_seconds() / 60
    duration_impact = session_order_data.groupby(
        pd.cut(session_order_data['Session Duration'],
               bins=[0, 15, 30, 45, 60, np.inf],
               labels=['0-15 mins', '16-30 mins', '31-45 mins', '46-60 mins', '60+ mins'])
    ).agg({
        'Amount (USD)': ['mean', 'count'],
        'Session Rating': 'mean'
    })


def visualize_insights(user_details, cooking_sessions, order_details):
    """
    Create visualizations for key insights
    """
    # Set up the plots
    plt.figure(figsize=(20, 15))
    plt.subplots_adjust(hspace=0.4, wspace=0.3)

    # 1. Age Group Distribution
    plt.subplot(2, 3, 1)
    user_details['Age Group'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('User Age Group Distribution')

    # 2. Total Orders by Location
    plt.subplot(2, 3, 2)
    order_details.groupby('Meal Type')['Amount (USD)'].sum().plot(kind='bar')
    plt.title('Total Revenue by Meal Type')
    plt.xticks(rotation=45)



    # 4. Session Duration vs Order Amount
    plt.subplot(2, 3, 4)
    sns.boxplot(x='Meal Type', y='Amount (USD)', data=order_details)
    plt.title('Order Amount by Meal Type')
    plt.xticks(rotation=45)

    # 5. User Registration Over Time
    plt.subplot(2, 3, 5)
    user_details['Registration Date'].dt.to_period('M').value_counts().sort_index().plot(kind='line')
    plt.title('User Registrations Over Time')

    # 6. Order Status Distribution
    plt.subplot(2, 3, 6)
    order_details['Order Status'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('Order Status Distribution')

    plt.suptitle('Cooking Platform Data Insights', fontsize=16)
    plt.tight_layout()
    plt.savefig('platform_insights.png')
    plt.close()

def generate_report(filename):
    """
    Load data and generate comprehensive analysis
    """
    # Load data
    user_details, cooking_sessions, order_details = load_and_clean_data(filename)

    # Perform analyses
    age_group_orders, location_orders = analyze_user_demographics(user_details, order_details)

    # Create visualizations
    visualize_insights(user_details, cooking_sessions, order_details)

    # Print reports
    print("=== User Demographics Analysis ===")
    print("\nOrders by Age Group:\n", age_group_orders)
    print("\nOrders by Location:\n", location_orders)

    print("\n=== Cooking Sessions Analysis ===")


    # Return key dataframes for further exploration
    return {
        'user_details': user_details,
        'cooking_sessions': cooking_sessions,
        'order_details': order_details,
        'age_group_orders': age_group_orders,
        'location_orders': location_orders,

    }

def business_recommendations(results):
    """
    Generate actionable business recommendations based on data analysis
    """
    recommendations = [
        "1. Target Marketing: Focus on the 26-35 age group, which shows the highest order frequency and average order value.",
        "2. Dish Optimization: Promote top-performing dishes like those identified in the popular dishes analysis, potentially creating special promotions.",
        "3. Session Duration Strategy: Encourage cooking sessions between 16-45 minutes, as they correlate with higher order ratings and values.",
        "4. Geographic Expansion: Consider targeted marketing in high-performing locations like New York and San Francisco.",
        "5. Meal Type Diversification: Create targeted offerings for less popular meal types to balance revenue streams.",
        "6. User Retention: Develop loyalty programs for users with high total order counts to encourage repeat business."
    ]

    print("\n=== Business Recommendations ===")
    for rec in recommendations:
        print(rec)

    return recommendations

def main():
    # Upload the Excel file
    filename = upload_excel_file()

    if filename:
        # Run the analysis
        results = generate_report(filename)

        # Generate recommendations
        final_recommendations = business_recommendations(results)

        # Save results to CSV
        results['age_group_orders'].to_csv('age_group_orders.csv')

        # Save recommendations to text file
        with open('business_recommendations.txt', 'w') as f:
            f.write("\n".join(final_recommendations))

        # Download generated files
        files.download('platform_insights.png')
        files.download('age_group_orders.csv')
        files.download('business_recommendations.txt')
    else:
        print("No file uploaded. Please upload the Excel file.")

# Run the main function
if __name__ == '__main__':
    main()

Please upload the Assignment.xlsx file


Saving Assignment.xlsx to Assignment (1).xlsx


<ipython-input-64-d5a494b3c412>:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_orders = user_order_analysis.groupby('Age Group')['Amount (USD)'].agg(['count', 'mean'])


=== User Demographics Analysis ===

Orders by Age Group:
            Total Orders  Average Order Value
Age Group                                   
18-25                 1                 13.0
26-35                11                 11.0
36-45                 4                 11.5
45+                   0                  NaN

Orders by Location:
                Total Orders  Average Order Value
Location                                        
Austin                    1            13.000000
Boston                    1            14.000000
Chicago                   3            10.666667
Los Angeles               3            10.333333
Miami                     1            11.000000
New York                  3            11.666667
San Francisco             2            10.750000
Seattle                   2            11.250000

=== Cooking Sessions Analysis ===

=== Business Recommendations ===
1. Target Marketing: Focus on the 26-35 age group, which shows the highest order frequency 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>